In [ ]:
import healpy as hp
import numpy as np
import pickle
import h5py
import altair as alt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cd ..

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
nside=512

In [ ]:
s4 = h5py.File("cmbs4_tophat.h5", mode='r', track_order=True)

In [ ]:
channels = [ch for ch in s4.keys() if "S" in ch]

In [ ]:
freqs = [s4[ch].attrs["center_frequency_GHz"] for ch in channels]

In [ ]:
print(freqs)

In [ ]:
# sort by frequency
channels = [x for _,x in sorted(zip(freqs,channels))]

In [ ]:
print(channels)

In [ ]:
alt.data_transformers.enable("default")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
import pysm3 as pysm
import so_pysm_models

from pysm3 import units as u

In [ ]:
import mapsims

In [ ]:
selection = {}
cl = {}
i_pol = 0

In [ ]:
config_file = "synchrotron_512.toml"
mapsim = mapsims.from_config(["common.toml", config_file])
component = config_file.split(".")[0].split("_")[0]
print(component)
filename = f"output/{nside}/C_ell/{component}.pkl"
with open(filename, "rb") as f:
    cl[component] = pickle.load(f)
if mapsim.pysm_components_string is not None:
    component_objects = [so_pysm_models.get_so_models(mapsim.pysm_components_string, nside=nside)]
else:
    component_objects = list(mapsim.pysm_custom_components.values())
sky = pysm.Sky(nside=nside,
               component_objects=component_objects,
               output_unit=u.uK_CMB
              )
onecl = pd.DataFrame(
    {ch:cl[component][ch][i_pol] for ch in channels}
).reset_index().rename(columns={"index":"ell"})
pysm_low = hp.anafast(
    sky.get_emission(s4[channels[0]].attrs["center_frequency_GHz"] * u.GHz),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[0]] = pysm_low

for ch in channels:
    m = hp.read_map(
   "../202006_reference_design/output/s4_reference_design_foregrounds/" + \
        f"SAT-{ch}_pole/cmbs4_KCMB_SAT-{ch}_pole_nside512_1_of_1.fits")
    sky_fraction = hp.mask_good(m).sum() / len(m) 
    onecl["output " + ch] = hp.anafast(m,
            lmax=onecl.ell.max()) * 1e12 / sky_fraction
    
pysm_high = hp.anafast(
        sky.get_emission(
            s4[channels[-1]].attrs["center_frequency_GHz"] * u.GHz,
        ),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[-1]] = pysm_high

onecl = onecl[onecl.ell != 0]

onecl_long = onecl.melt("ell", var_name="channel")
onecl_long[onecl_long.value == 0] = np.nan
selection[component] = alt.selection_multi(fields=['channel'], bind="legend")

alt.Chart(onecl_long, title=component).mark_line().encode(
    x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
    y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
    color=alt.Color('channel', 
        sort=["PySM " + channels[0]] + channels + ["PySM " + channels[-1]]
                   ),
    opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
).add_selection(selection[component]).interactive()

In [ ]:
config_file = "dust_512.toml"
mapsim = mapsims.from_config(["common.toml", config_file])
component = config_file.split(".")[0].split("_")[0]
print(component)
filename = f"output/{nside}/C_ell/{component}.pkl"
with open(filename, "rb") as f:
    cl[component] = pickle.load(f)
if mapsim.pysm_components_string is not None:
    component_objects = [so_pysm_models.get_so_models(mapsim.pysm_components_string, nside=nside)]
else:
    component_objects = list(mapsim.pysm_custom_components.values())
sky = pysm.Sky(nside=nside,
               component_objects=component_objects,
               output_unit=u.uK_CMB
              )


    
onecl = pd.DataFrame(
    {ch:cl[component][ch][i_pol] for ch in channels}
).reset_index().rename(columns={"index":"ell"})
pysm_low = hp.anafast(
    sky.get_emission(s4[channels[0]].attrs["center_frequency_GHz"] * u.GHz),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[0]] = pysm_low
for ch in channels:
    m = hp.read_map(
   "../202006_reference_design/output/s4_reference_design_foregrounds/" + \
        f"SAT-{ch}_pole/cmbs4_KCMB_SAT-{ch}_pole_nside512_1_of_1.fits")
    sky_fraction = hp.mask_good(m).sum() / len(m) 
    onecl["output " + ch] = hp.anafast(m,
            lmax=onecl.ell.max()) * 1e12 / sky_fraction
pysm_high = hp.anafast(
        sky.get_emission(
            s4[channels[-1]].attrs["center_frequency_GHz"] * u.GHz,
        ),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[-1]] = pysm_high

onecl = onecl[onecl.ell != 0]
onecl_long = onecl.melt("ell", var_name="channel")
onecl_long[onecl_long.value == 0] = np.nan

selection[component] = alt.selection_multi(fields=['channel'], bind='legend')

alt.Chart(onecl_long, title=component).mark_line().encode(
    x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
    y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
    color=alt.Color('channel', 
        sort=["PySM " + channels[0]] + channels + ["PySM " + channels[-1]]
                   ),
    opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
).add_selection(selection[component]).interactive()

In [ ]:
config_file = "cmb.toml"
mapsim = mapsims.from_config(["common.toml", config_file])
component = config_file.split(".")[0].split("_")[0]
print(component)
filename = f"output/{nside}/C_ell/{component}.pkl"
with open(filename, "rb") as f:
    cl[component] = pickle.load(f)
if mapsim.pysm_components_string is not None:
    component_objects = [so_pysm_models.get_so_models(mapsim.pysm_components_string, nside=nside)]
else:
    component_objects = list(mapsim.pysm_custom_components.values())
sky = pysm.Sky(nside=nside,
               component_objects=component_objects,
               output_unit=u.uK_CMB
              )
onecl = pd.DataFrame(
    {ch:cl[component][ch][i_pol] for ch in channels}
).reset_index().rename(columns={"index":"ell"})
pysm_low = hp.anafast(
    sky.get_emission(s4[channels[0]].attrs["center_frequency_GHz"] * u.GHz),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[0]] = pysm_low

for ch in channels:
    m = hp.read_map(
   "../202006_reference_design/output/s4_reference_design_cmb_r0/" + \
        f"SAT-{ch}_pole/cmbs4_KCMB_SAT-{ch}_pole_nside512_1_of_1.fits")
    sky_fraction = hp.mask_good(m).sum() / len(m) 
    onecl["output " + ch] = hp.anafast(m,
            lmax=onecl.ell.max()) * 1e12 / sky_fraction

pysm_high = hp.anafast(
        sky.get_emission(
            s4[channels[-1]].attrs["center_frequency_GHz"] * u.GHz,
        ),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[-1]] = pysm_high

onecl = onecl[onecl.ell != 0]
onecl_long = onecl.melt("ell", var_name="channel")
onecl_long[onecl_long.value == 0] = np.nan

selection[component] = alt.selection_multi(fields=['channel'], bind='legend')

alt.Chart(onecl_long, title=component).mark_line().encode(
    x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
    y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
    color=alt.Color('channel', 
        sort=["PySM " + channels[0]] + channels + ["PySM " + channels[-1]]
                   ),
    opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
).add_selection(selection[component]).interactive()

In [ ]:
config_file = "cmb_tensor.toml"
mapsim = mapsims.from_config(["common.toml", config_file])
component = config_file.split(".")[0]
print(component)
filename = f"output/{nside}/C_ell/{component}.pkl"
with open(filename, "rb") as f:
    cl[component] = pickle.load(f)
if mapsim.pysm_components_string is not None:
    component_objects = [so_pysm_models.get_so_models(mapsim.pysm_components_string, nside=nside)]
else:
    component_objects = list(mapsim.pysm_custom_components.values())
sky = pysm.Sky(nside=nside,
               component_objects=component_objects,
               output_unit=u.uK_CMB
              )
onecl = pd.DataFrame(
    {ch:cl[component][ch][i_pol] for ch in channels}
).reset_index().rename(columns={"index":"ell"})
pysm_low = hp.anafast(
    sky.get_emission(s4[channels[0]].attrs["center_frequency_GHz"] * u.GHz),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[0]] = pysm_low

for ch in channels:
    m = hp.read_map(
   "../202006_reference_design/output/s4_reference_design_cmb_tensor_only_r3e-3/" + \
        f"SAT-{ch}_pole/cmbs4_KCMB_SAT-{ch}_pole_nside512_1_of_1.fits")
    sky_fraction = hp.mask_good(m).sum() / len(m) 
    onecl["output " + ch] = hp.anafast(m,
            lmax=onecl.ell.max()) * 1e12 / sky_fraction
    
pysm_high = hp.anafast(
        sky.get_emission(
            s4[channels[-1]].attrs["center_frequency_GHz"] * u.GHz,
        ),
lmax=onecl.ell.max())[i_pol]
onecl["PySM " + channels[-1]] = pysm_high

onecl = onecl[onecl.ell != 0]
onecl_long = onecl.melt("ell", var_name="channel")
onecl_long[onecl_long.value == 0] = np.nan

selection[component] = alt.selection_multi(fields=['channel'], bind='legend')

alt.Chart(onecl_long, title=component).mark_line().encode(
    x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
    y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
    color=alt.Color('channel', 
        sort=["PySM " + channels[0]] + channels + ["PySM " + channels[-1]]
                   ),
    opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
).add_selection(selection[component]).interactive()